In [1]:
import tensorflow
import numpy as np
import pandas as pd
import tensorflow.keras as keras
from skimage import io
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import io
from skimage import measure 
from skimage import feature
from skimage.morphology import disk
from skimage.filters import rank
from skimage import exposure
import warnings
warnings.filterwarnings("ignore")
marks = pd.read_csv('D:/Kaggle/train_ship_segmentations.csv') # Markers for ships
images = os.listdir('D:/Kaggle/train_images') # Images for training
test_marks = pd.read_csv('D:/Kaggle/test_ship_segmentations.csv')
test_images = os.listdir('D:/Kaggle/test_images')
os.chdir("D:/Kaggle/train_images")

In [2]:
AUGMENT_BRIGHTNESS = False

In [3]:
def mask_part(pic):
    '''
    Function that encodes mask for single ship from .csv entry into numpy matrix
    '''
    back = np.zeros(768**2)
    starts = pic.split()[0::2]
    lens = pic.split()[1::2]
    for i in range(len(lens)):
        back[(int(starts[i])-1):(int(starts[i])-1+int(lens[i]))] = 1
    return np.reshape(back, (768, 768, 1))

def is_empty(key):
    '''
    Function that checks if there is a ship in image
    '''
    df = train_data[train_data['ImageId'] == key].iloc[:,1]
    if len(df) == 1 and type(df.iloc[0]) != str and np.isnan(df.iloc[0]):
        return True
    else:
        return False
    
def masks_all(key):
    '''
    Merges together all the ship markers corresponding to a single image
    '''
    df = train_data[train_data['ImageId'] == key].iloc[:,1]
    masks= np.zeros((768,768,1))
    if is_empty(key):
        return masks
    else:
        for i in range(len(df)):
            masks += mask_part(df.iloc[i])
        return np.transpose(masks, (1,0,2))

In [4]:
marks['ships'] = marks['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
train_data = marks[marks["ships"]!=0]
train_ids = list(train_data["ImageId"])
no_ships_train = list(set(images) & set(train_ids))
train_images_ids = list(set(no_ships_train) & set(images))

In [5]:
test_marks['ships'] = test_marks['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
test_data = test_marks[test_marks["ships"]!=0]
test_ids = list(test_data["ImageId"])
no_ships_test = list(set(test_images) & set(test_ids))
test_images_ids = list(set(no_ships_test) & set(test_images))

In [ ]:
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.util.montage import montage2d as montage
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)

In [6]:
def Gen_clear_data(ids, batch_size):    
    X = np.zeros((batch_size, 768, 768, 3))
    Y = np.zeros((batch_size, 768, 768, 1))
    while True:
        print("hello")
        for i in range(batch_size):
            index = np.random.choice(len(ids))
            print(ids[index])
            X[i] = io.imread(ids[index])/255.0
            Y[i] = masks_all(ids[index])
        yield X,Y
            
def Gen_aug_data(batch, batch_size, seed = None):

    dg_args = dict(featurewise_center = False, 
              samplewise_center = False,
              rotation_range = 15, 
              width_shift_range = 0.1, 
              height_shift_range = 0.1, 
              shear_range = 0.01,
              zoom_range = [0.9, 1.25],  
              horizontal_flip = True, 
              vertical_flip = True,
              fill_mode = 'reflect',
              data_format = 'channels_last')
    if AUGMENT_BRIGHTNESS:
        dg_args[' brightness_range'] = [0.5, 1.5]
    image_gen = ImageDataGenerator(**dg_args)

    if AUGMENT_BRIGHTNESS:
        dg_args.pop('brightness_range')
    label_gen = ImageDataGenerator(**dg_args)
    np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
    for i in range(batch_size):
        seed = np.random.choice(range(9999))
        aug_x = image_gen.flow(255*batch[0], batch_size = batch_size, seed = seed, shuffle=True)
        aug_y = label_gen.flow(batch[1],batch_size = batch_size, seed = seed, shuffle=True)

        yield next(aug_x)/255.0, next(aug_y)

class Generator():
    def __init__(self, ids, batch_size,augmentation = False, add_feautures = False):
        self.ids = ids
        self.batch_size = batch_size
        self.augmentation = augmentation
        self.add_feautures = add_feautures
        self.AUGMENT_BRIGHTNESS = False
        self.batch_ids = []
           
    def Data_generator(self):
        while True:
            if self.augmentation == False:
                gen = Gen_clear_data(self.ids, self.batch_size)
                yield next(gen)
            elif self.augmentation == True:
                gen = Gen_clear_data(self.ids,self.batch_size)
                data = next(gen)
                gen_aug_data = Gen_aug_data(data,self.batch_size)
                yield next(gen_aug_data)


In [7]:
ddd = Gen_clear_data(train_ids, 3)

In [8]:
for i in range(10):
    
    tup = next(ddd)

hello
663b10d81.jpg
e3169b14a.jpg
11ef28b6d.jpg
hello
be3abb90c.jpg
69bd1b7dc.jpg
5f92de2d0.jpg
hello
b25bc8ca4.jpg
40951b370.jpg
cdcea0e9f.jpg
hello
b418edd67.jpg
345c4036d.jpg
1e4722857.jpg
hello
54f26faa5.jpg
8421eaa36.jpg
d36d886a3.jpg
hello
d6bc1a321.jpg
f3bb38ccc.jpg
706ef5f93.jpg
hello
8dbce7878.jpg
e45e9beef.jpg
d4a41ec48.jpg
hello
96b0a4657.jpg
f6f75def4.jpg
4b510d261.jpg
hello
d8f776013.jpg
6046848af.jpg
c65c40baa.jpg
hello
e44ce8314.jpg
ce536a86a.jpg
74acc425d.jpg


In [ ]:
tup[1].shape

In [ ]:
t_x, t_y = a[0], a[1]
print('x', t_x.shape, t_x.dtype, t_x.min(), t_x.max())
print('y', t_y.shape, t_y.dtype, t_y.min(), t_y.max())
# only keep first 9 samples to examine in detail
t_x = t_x[:9]
t_y = t_y[:9]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))
ax1.imshow(montage_rgb(t_x), cmap='gray')
ax1.set_title('images')
ax2.imshow(montage(t_y[:, :, :, 0]), cmap='gray_r')
ax2.set_title('ships')

In [ ]:
# Intersection over Union for Objects
def IoU(y_true, y_pred, tresh=1e-10):
    Intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    Union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - Intersection
    return K.mean( (Intersection + tresh) / (Union + tresh), axis=0)
# Intersection over Union for Background
def back_IoU(y_true, y_pred):
    return IoU(1-y_true, 1-y_pred)
# Loss function
def IoU_loss(in_gt, in_pred):
    #return 2 - back_IoU(in_gt, in_pred) - IoU(in_gt, in_pred)
    return 1 - IoU(in_gt, in_pred)

In [ ]:
from skimage import io
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K

In [ ]:
inputs = Input((768, 768, 3))

c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (inputs)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)

u5 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c4)
u5 = concatenate([u5, c3])
c5 = Conv2D(32, (3, 3), activation='relu', padding='same') (u5)
c5 = Conv2D(32, (3, 3), activation='relu', padding='same') (c5)

u6 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c2])
c6 = Conv2D(16, (3, 3), activation='relu', padding='same') (u6)
c6 = Conv2D(16, (3, 3), activation='relu', padding='same') (c6)

u7 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c1], axis=3)
c7 = Conv2D(8, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(8, (3, 3), activation='relu', padding='same') (c7)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c7)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer="adam", loss= IoU_loss, metrics=[IoU, back_IoU])
model.summary()

In [ ]:
Gen = Generator(train_images_ids,batch_size=3, augmentation=False)
generator = Gen.Data_generator()

In [ ]:
results = model.fit_generator(generator=generator,steps_per_epoch=10,epochs = 10)